In [16]:
import os
from os import path
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import librosa
from magenta.models.nsynth.reader import NSynthDataset

%matplotlib inline

In [3]:
!ls /corpora

corrupted		 maps_music_transcription  nsynth	urbansound8k
freesound-audio-tagging  midi_classic_piano	   oxford_pets	voices
gtzan_genre		 million_song_dataset	   sitw		voxceleb
gtzan_music_speech	 mnist			   speech	voxforge
maestro			 music_net		   text		youtube8m


In [4]:
train_record_fn = "/corpora/nsynth/nsynth-train.tfrecord"
assert path.exists(train_record_fn)

In [18]:
ds = NSynthDataset(train_record_fn, is_training=True)
batch = ds.get_example(1)
batch

{'audio': <tf.Tensor 'ParseSingleExample_1/ParseSingleExample:0' shape=(64000,) dtype=float32>,
 'instrument_family': <tf.Tensor 'ParseSingleExample_1/ParseSingleExample:1' shape=(1,) dtype=int64>,
 'instrument_source': <tf.Tensor 'ParseSingleExample_1/ParseSingleExample:2' shape=(1,) dtype=int64>,
 'note_str': <tf.Tensor 'ParseSingleExample_1/ParseSingleExample:3' shape=() dtype=string>,
 'pitch': <tf.Tensor 'ParseSingleExample_1/ParseSingleExample:4' shape=(1,) dtype=int64>,
 'qualities': <tf.Tensor 'ParseSingleExample_1/ParseSingleExample:5' shape=(10,) dtype=int64>,
 'velocity': <tf.Tensor 'ParseSingleExample_1/ParseSingleExample:6' shape=(1,) dtype=int64>}

In [19]:
with tf.Session(config=tf.ConfigProto(device_count={'GPU': 0})) as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    batch = sess.run(batch)
    print(batch.keys())

['instrument_family', 'instrument_source', 'note_str', 'qualities', 'pitch', 'velocity', 'audio']
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Run call was cancelled
